In [18]:
from ASmartis import *   
from Buttons import *   
from Chart2 import *
from ipywidgets import HBox, VBox, Layout
from IPython.display import display, clear_output
#from math import pi, sin, cos

class ElektronImFeld(ASmartis):
    
    def __init__(self,parameterList):        
        
        self.oldElectronPosition = [10,150,5]
        self.oldElectronVelocity = [0,0,0]
        super().__init__(parameterList)
        #self.updateTime = 0.08
        
    
    def createButtons(self):
        
        self.buttonsObject = Buttons(self)
        
        self.buttonsObject.newTogglePlayPause("b_play", 40,40)
        self.buttonsObject.newReset("b_reset", 40,40)  
        self.buttonsObject.newFloatSlider("s_velocity", 0,100,1)
        self.buttonsObject.changeDescription("s_velocity","$v_{x}$")
        self.buttonsObject.newRadioButtons("b_field", ["E","B"], "Feldtyp") 
        self.buttonsObject.newIntSlider("s_field", -10,10,0)
        self.buttonsObject.changeDescription("s_field","Feldstärke")
        
        self.buttonStatesDict = self.buttonsObject.getStates()
        self.oldButtonStatesDict = dict(self.buttonStatesDict)
        
        
            
    def createChart(self):

        # Ebenen definieren
        layers = ["background", "field", "path", "elektron"]
        self.chartWidth = 700
        self.chartHeight = 300
        self.chartObject = Chart(layers, self.chartWidth, self.chartHeight)
        
        self.x_fieldStart = 100
        self.y_fieldStart = 10        
        self.fieldWidth = 500
        self.fieldHeight = 280         
        
        ##Background
        currentLayer = "background"
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#c0c0c0")
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#ffffff")
        self.chartObject.changeLayerLook(currentLayer,"lineDash",[3,3])
        self.chartObject.drawRectangle(currentLayer, self.x_fieldStart, self.y_fieldStart, self.fieldWidth, self.fieldHeight,0)   
        
        ##Electron
        currentLayer = "elektron"
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#ffff00")
        self.chartObject.clearLayer(currentLayer)     
        
        ##Path
        currentLayer = "path"
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#f0f0f0")
        self.chartObject.changeLayerLook(currentLayer,"lineWidth",2)
        
        self.chartObject.update("background","elektron","path") 
        
        self.drawElectron()
        self.updateField()

    def visualize(self): 
        
        self.buttonsDict = self.buttonsObject.getButtons()
        self.chart = self.chartObject.getChart()
      
        display(HBox([self.chart,self.buttonsDict["b_play"],self.buttonsDict["b_reset"]]))
        display(HBox([self.buttonsDict["b_field"],VBox([self.buttonsDict["s_velocity"],self.buttonsDict["s_field"]])]))               
        
    def update(self):     
                
        if not self.buttonStatesDict["b_reset"]:
            if (self.oldButtonStatesDict["b_field"] != self.buttonStatesDict["b_field"] or
                self.oldButtonStatesDict["s_field"] != self.buttonStatesDict["s_field"]):
                self.updateField()
            if self.buttonStatesDict["b_play"]:
                self.updateElectronPosition()
                self.drawElectron()
        else:
            self.reset()
        
    def updateField(self):
        
        currentLayer = "field"
        currentField = self.buttonStatesDict["b_field"]
        currentFieldStrength = int(self.buttonStatesDict["s_field"]*2)
        
        self.chartObject.clearLayer(currentLayer)
        
        if currentField == "B":
            self.chartObject.changeLayerLook(currentLayer,"lineColor","#00d000")
            self.chartObject.changeLayerLook(currentLayer,"fillingColor","#00d000")
        else:
            self.chartObject.changeLayerLook(currentLayer,"lineColor","#e8082c")
            self.chartObject.changeLayerLook(currentLayer,"fillingColor","#e8082c")
        
        if currentFieldStrength>= 0:
            self.chartObject.drawParallelArrows(currentLayer,self.x_fieldStart,self.y_fieldStart,
                self.fieldWidth, self.fieldHeight, currentFieldStrength, "vertical",10,10)
        else:
            self.chartObject.drawParallelArrows(currentLayer,self.x_fieldStart,
                self.y_fieldStart+self.fieldHeight,-self.fieldWidth, -self.fieldHeight, 
                currentFieldStrength, "vertical",10,10)
            
        self.chartObject.update(currentLayer)

    
    def drawElectron(self):
        
        currentElectronLayer = "elektron"
        currentPathLayer = "path"
        
        xPosition = self.oldElectronPosition[0]
        yPosition = self.oldElectronPosition[1]
        zPosition = self.oldElectronPosition[2]
        
        self.chartObject.drawCircle(currentElectronLayer,xPosition, yPosition, zPosition,0)  
        self.chartObject.drawCircle(currentPathLayer,xPosition, yPosition,1,1)
        self.chartObject.update(currentElectronLayer,currentPathLayer)
        
        
    
    def updateElectronPosition(self):

        currentField = self.buttonStatesDict["b_field"]
        currentFieldStrength = self.buttonStatesDict["s_field"]
        sliderVelocity = self.buttonStatesDict["s_velocity"]
        oldXPosition = self.oldElectronPosition[0]
        oldYPosition = self.oldElectronPosition[1]
        oldZPosition = self.oldElectronPosition[2]
        oldXVelocity = self.oldElectronVelocity[0]
        oldYVelocity = self.oldElectronVelocity[1]
        oldZVelocity = self.oldElectronVelocity[2]
        t = self.updateTime
                
        if currentField == "E":
            newXPosition = float(oldXPosition) + t*sliderVelocity
            newZVelocity = oldZVelocity
            newZPosition = oldZPosition+oldZVelocity*t            
            
            if newXPosition > self.x_fieldStart and newXPosition <(self.x_fieldStart+self.fieldWidth):
                newYVelocity = oldYVelocity-currentFieldStrength*t
                newYPosition = float(oldYPosition) + (newYVelocity*t)
            else:
                newYVelocity = oldYVelocity
                newYPosition = oldYPosition+t*newYVelocity     
                
        elif currentField == "B":
            newXPosition = float(oldXPosition) + t*sliderVelocity
            newYVelocity = oldYVelocity
            newYPosition = oldYPosition+t*newYVelocity     
            
            if newXPosition > self.x_fieldStart and newXPosition <(self.x_fieldStart+self.fieldWidth):
                newZVelocity = oldZVelocity+t*sliderVelocity*currentFieldStrength/1000
                newZPosition = float(oldZPosition)+(newZVelocity*t)
            else:
                newZVelocity = oldZVelocity
                newZPosition = oldZPosition+oldZVelocity*t
            
            
        if (newXPosition > self.chartWidth or newYPosition < self.y_fieldStart or 
            newYPosition > (self.y_fieldStart+self.fieldHeight) or newZPosition < 1):
            
            newXPosition = 10
            newYPosition = 150
            newZPosition = 5
            newYVelocity = 0
            newZVelocity = 0
            self.buttonsObject.changeState("b_play",False)
                

        
        self.oldElectronPosition[0] = newXPosition
        self.oldElectronPosition[1] = newYPosition
        self.oldElectronPosition[2] = newZPosition      
        self.oldElectronVelocity[1] = newYVelocity
        self.oldElectronVelocity[2] = newZVelocity
        
        

    def reset(self):        
        
        self.buttonsObject.changeState("b_play",False)
        self.buttonsObject.changeState("s_velocity",1)
        self.buttonsObject.changeState("s_field",0)
        self.buttonsObject.changeState("b_field","E")
        self.oldElectronPosition = [10,150,5]
        self.oldElectronVelocity = [0,0,0]
                
        self.drawElectron()        
        self.updateField()
        
        currentLayer = "path"
        self.chartObject.clearLayer(currentLayer)
        self.chartObject.update(currentLayer) 
        
        self.buttonsObject.changeState("b_reset",False)
        
        

    


In [20]:
e = ElektronImFeld(None)
e.visualize()
e.start()


In [11]:
if __name__ == '__main__' and '__file__' in globals():
    e.stop()
    del e